<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-07-20 15:37:35--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11848 (12K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-07-20 15:37:35 (92.9 MB/s) - ‘utils.py.1’ saved [11848/11848]



In [2]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'data')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68

In [6]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 20092 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [8]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 21.15
black     	=> 16.27
blue     	=> 18.23
brown     	=> 21.13
cream     	=> 22.32
crimson     	=> 22.52
gold     	=> 22.32
green     	=> 24.99
grey     	=> 14.63
navy-blue     	=> 23.09
orange     	=> 26.37
red     	=> 19.32
silver     	=> 23.78
titanium     	=> 22.32
white     	=> 3.5
yellow     	=> 24.38


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [11]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [12]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
628/628 [==============================] - 634s 976ms/step - loss: 11.6381 - auc: 0.8527 - accuracy: 0.7673 - val_loss: 0.6813 - val_auc: 0.8725 - val_accuracy: 0.7854
Epoch 2/15
628/628 [==============================] - 269s 426ms/step - loss: 6.7064 - auc: 0.9346 - accuracy: 0.8594 - val_loss: 0.6945 - val_auc: 0.8704 - val_accuracy: 0.7894
Epoch 3/15
628/628 [==============================] - 268s 426ms/step - loss: 4.7788 - auc: 0.9607 - accuracy: 0.8970 - val_loss: 0.8397 - val_auc: 0.8753 - val_accuracy: 0.8120
Epoch 4/15
628/628 [==============================] - 267s 424ms/step - loss: 3.6380 - auc: 0.9746 - accuracy: 0.9248 - val_loss: 0.8163 - val_auc: 0.8442 - val_accuracy: 0.7828
Epoch 5/15
628/628 [==============================] - 269s 426ms/step - loss: 2.7398 - auc: 0.9827 - accuracy: 0.9375 - val_loss: 0.6579 - val_auc: 0.8964 - val_accuracy: 0.8363
Epoch 6/15
628/628 [==============================] - 273s 432ms/step - loss: 2.0308 - auc: 0.9888 - accuracy

In [13]:
!cp ./logs/checkpoints/RGB-model.h5 ./drive/MyDrive/checkpoints/rgb-model-aug.h5